<h2> Welcome </h2>

I try to make prediction price, data cleaning and feature engineering in this kenel. if you catch my error, please warn me :)

**Prediction Result;**

**Root Mean Squered Error : 28 ± 1** 


If you like it and want to support, please vote. This will increase my motivation :)


In [ ]:
#load fundamental libraary
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#load data
dataset = pd.read_csv("../input/listings.csv")
dataset2 = pd.read_csv('../input/listings_summary.csv')

Now, We oveview datasets and get significant features according to us. 

In [ ]:
dataset.head(1)

In [ ]:
#get significant features for according to me from this dataset and show 1 example
important_features = ["room_type","reviews_per_month","number_of_reviews","minimum_nights","calculated_host_listings_count","availability_365","longitude","latitude","price"]
dataset = dataset[important_features]
dataset.head(1)

In [ ]:
dataset.info()

In [ ]:
#look at nulls
fig, ax = plt.subplots(figsize = (9, 5))
sns.heatmap(dataset.isnull(), cmap = "cubehelix_r", yticklabels='')
plt.show()

* **There are a lot of nan values in reviews_per_month. It is more meaningful than number_of_reviews a bit but There aren't a lot of diffirence. I will delete reviews_per_month.**

In [ ]:
del dataset["reviews_per_month"]


Now, There is no null data in listining.csv. It is good news for us :)

Let's take a look dataset2.

In [ ]:
dataset2.head(1)

In [ ]:
dataset2.info()

In [ ]:
#look at nulls
fig, ax = plt.subplots(figsize = (20, 5))
sns.heatmap(dataset2.isna(), cmap = "cubehelix_r", yticklabels='')
plt.show()

* **There are too many null values in some feature. We can ignore feature that contains too many nulls**

In [ ]:
#ignore features that contain more than %30 
cols = dataset2.columns.values
lenght = dataset2.shape[0]
col_list = [] 
for i in cols:
    x = dataset2[i]
    rate = round(x.isna().sum()/lenght,3)
    if rate < 0.30:
        col_list.append(i)
        print(i,"contain :%",100*rate ," ok :)")
    else:
        print(i,"contain :%",rate*100 ," fail !!")

In [ ]:
#selectable features
print("Selectable feature count : ",len(col_list))

In [ ]:
#get important feature
get_features = ["cancellation_policy","amenities","availability_365","requires_license","instant_bookable",
                "guests_included","extra_people","review_scores_rating","require_guest_phone_verification",
               "bathrooms","bedrooms","beds","bed_type","accommodates","host_total_listings_count","host_has_profile_pic",
               "host_identity_verified","is_location_exact","property_type","host_is_superhost","room_type","maximum_nights","minimum_nights"]
dataset2 = dataset2[get_features]
dataset2.head()

In [ ]:
dataset2.info()

<h4>What We learn from info ?</h4>

* We have 22552 sample. 

Feature contains null;

 <p>**#####** => Contains too much Nan values</p>
 <p>**#**     => Contains too few Nan values</p>

* review_scores_rating #####
* bathrooms #
* bedrooms #
* beds #
* host_has_profile_pic #
* host_identity_verified #
* host_total_listings_count #




* **We need handle this feature. If feature contains too low nan values, I fill it with majority values of feature. Else I  will using diffirent teqniques.**

* **We can create new features from exist features. Example; We can create distance from latitude and longtitude. This feature response this question. "How far is Home/Flat to Berlin ?"** 

Let's Start!!

<h3>Working on listings.csv dataset</h3>

In [ ]:
_dataset = dataset.copy()

In [ ]:
#create distance feature from latitude and longtitude 
from geopy.distance import great_circle
def distance_from_berlin(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    record = (lat, lon)
    return great_circle(berlin_centre, record).km

#add distanse dataset
_dataset['distance'] = _dataset.apply(lambda x: distance_from_berlin(x.latitude, x.longitude), axis=1)

del _dataset['latitude']
del _dataset['longitude']
_dataset.head(1)

In [ ]:
#nominal_categorical
print(_dataset.room_type.unique())
x = _dataset[["room_type"]]
x.room_type = pd.Categorical(x['room_type'])
del _dataset['room_type']
dummies_room_type = pd.get_dummies(x, prefix = 'room_type')
_dataset = pd.concat([_dataset,dummies_room_type], axis=1)
_dataset.head(3)

<h2>Working on listings_summary dataset</h2> 

In [ ]:
_dataset2 = dataset2.copy()

In [ ]:
_dataset2.head()

In [ ]:
#del same features
col_list2 = list(_dataset.columns.values)
col_list2.append("room_type")
for i in col_list2:
    if i in _dataset2.columns.values:
        print("deleted ",i," feature!")
        del _dataset2[i]

In [ ]:
#convert string true and false to numeric
def object2bool(x):
    if x=="t" or x=="T":
        return 1.0
    elif x=="f" or x=="F":
        return 0.0
    else:
        return None

boolean_features = ["require_guest_phone_verification","host_is_superhost","host_has_profile_pic",
                    "host_identity_verified","is_location_exact","requires_license","instant_bookable"]

for i in boolean_features:
    _dataset2[i] = _dataset2[i].map(object2bool)

_dataset2.head()

In [ ]:
#nominal_categorical bed_type and property_type
for i in ["bed_type","property_type","cancellation_policy"]:
    x = _dataset2[[i]]
    x.room_type = pd.Categorical(x[i])
    del _dataset2[i]
    dummies = pd.get_dummies(x, prefix = i)
    _dataset2 = pd.concat([_dataset2,dummies], axis=1)

_dataset2.head(3)

In [ ]:
#amenities count
def amenities_counter(x):
    return len(x.split(","))

_dataset2.amenities = _dataset2.amenities.map(amenities_counter)
_dataset2.head(3)

In [ ]:
#clean $ sign
def clean_price(x):
    return float(x.replace("$",""))

_dataset2.extra_people = _dataset2.extra_people.map(clean_price)
_dataset2.head(3)

In [ ]:
_dataset2 = _dataset2.astype(float)
_dataset = _dataset.astype(float)

* **We should fill nan values. I will fill nan values with median.**

In [ ]:
#get features that contain nulls
null_cols = []
for i in _dataset2.columns.values:
    if _dataset2[i].isna().sum()>0:
        null_cols.append(i)

print("Features that contain nulls;  \n\n",null_cols)

In [ ]:
#fill nulls with median
for i in null_cols:
    median = _dataset2[i].median()
    _dataset2[i] = _dataset2[i].fillna(median)

In [ ]:
print("Dataset2 null count :",_dataset2.isna().any().sum())
print("Dataset null count :",_dataset.isna().any().sum())

<h3>OK :) There is no Null data and all feature transformed to numeric.</h3>

Now, We can concat dataset and dataset2.

In [ ]:
#concat
data = pd.concat([_dataset,_dataset2],axis=1)
data.shape

In [ ]:
data.describe()

<h2>Clean Outlier with Z-score</h2>

In [ ]:
#outlier detection with z score
def detect_outlier(data_1):
    feature_outliers=[]
    threshold=7
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    
    counter=0
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            feature_outliers.append(counter)
        counter += 1
    return feature_outliers

In [ ]:
outliers = np.array([])
print("--Feature and Outlier Counts--\n")
for i in data.columns:
    f_out = detect_outlier(data[i])
    outliers = np.concatenate((outliers,np.asarray(f_out)))
    print(i ," outlier count :",len(f_out))
                              
outliers = np.unique(outliers,0)
print("Total Unique Outlier Index Count:",len(outliers))

In [ ]:
clean_data = data.drop(outliers,axis=0)
clean_data.describe()

In [ ]:
f, ax = plt.subplots( figsize=(35,35) )
sns.heatmap(data.corr(),annot=True,linewidths=1)
plt.show()

* **Nice correalation matrix. There is no too much coraleted feature. **

<h2>Modeling</h2>

In [ ]:
#split data test and train
from sklearn.model_selection import train_test_split

X = clean_data.drop(["price"],axis=1)
y = clean_data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=42)

In [ ]:
# normalize data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.metrics import mean_squared_error
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks.callbacks import ModelCheckpoint

#set parameters
batch_size = 32
epochs = 25

model = Sequential()
model.add(Dense(512,input_shape=(data.shape[1]-1,), activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation= 'relu'))
model.add(Dense(1, activation= 'relu'))

filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss = mean_squared_error,
              optimizer = Adam(),
              metrics=['mean_squared_error'])

hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test),callbacks=callbacks_list)

In [ ]:
#plot training history
figsize=(10,5)
ax,_ = plt.subplots(figsize=figsize)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('modelloss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upperleft')
plt.show()

In [ ]:
ls

In [ ]:
import os
import copy

saved_loss_file = None
loss_min = 100000
# r=root, d=directories, f = files
for r, d, f in os.walk('./'):
    for file in f:
        if '.hdf5' in file:
            filename = os.path.join(r, file)
            loss = int(filename.split('-')[3].split('.')[0])
            if loss < loss_min:
                loss_min = loss
                saved_loss_file = filename
                
print("Saved min loss : ",loss_min,"\nFile :",saved_loss_file)

In [ ]:
#load model
from keras.models import load_model
best_model = load_model(saved_loss_file)

In [ ]:
#prediction
pred_mlp = best_model.predict(X_test)

In [ ]:
#evaluate
print('\n# Model Evaluate')
results = best_model.evaluate(X_test, y_test)
print('Test Mse Loss:', results[0])

RMSE = np.sqrt(results[0])
print("RMSE:",RMSE)


Thanks for read this kernal :) 

If you have suggestion for me, please write comment. I want to improve myself.

See You :)